Make the data frame that classifies the residues that are part of the interface 3 different types of classification methods will be used, first of all the the residues predicted above the threshold MCC 0.345 are classified as protein interface residues by the PIPENN-EMB model. Also the sequence neighbours will be used to analyse whether these can shine light on new interesting protein residues and also structural neighbouringing residues within a 5A radius will be used

In [ ]:
# #Code to get the AF2 structures from the AF2 database
# from posixpath import split
# import requests
# import os
# import pandas as pd
# import Bio

# def download_alphafold_pdb(uniprot_id, save_directory):
#     """Download AlphaFold PDB file for a given UniProt ID."""
#     url = f'https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()
#         # Check if data is a list and not empty
#         if isinstance(data, list) and len(data) > 0:
#             pdb_url = data[0].get('pdbUrl')  # Access the first item and get pdbUrl
#             if pdb_url:
#                 save_path = os.path.join(save_directory, f"{uniprot_id}.pdb")
#                 pdb_response = requests.get(pdb_url)
#                 if pdb_response.status_code == 200:
#                     with open(save_path, 'wb') as file:
#                         file.write(pdb_response.content)
#                     print(f"PDB file saved for {uniprot_id} to {save_path}")
#                 else:
#                     print(f"Failed to download PDB file for {uniprot_id}.")
#             else:
#                 print(f"No PDB URL found for {uniprot_id}.")
#         else:
#             print(f"No data found for UniProt ID {uniprot_id}.")
#     else:
#         print(f"Failed to retrieve data for UniProt ID {uniprot_id}. Status code: {response.status_code}")

# # Ensure the save directory exists
# os.makedirs(save_directory, exist_ok=True)

# # Specify your list of UniProt IDs and the directory to save the files
# mutations_all_proteins_df = pd.read_csv('/Users/davidthomas/Desktop/Research_internship/Determining_interesting_mutations/merged_df_non_sensitive_mutationcounts_all_proteins.csv')
# uniprot_id_list = mutations_all_proteins_df['uniprot_id'].unique().tolist()
# save_directory = "/Users/davidthomas/Desktop/Research_internship/Determining_interesting_mutations/AF2_structures"  # Update with your desired directory

# # Loop through each UniProt ID and download its PDB file
# for uniprot_id in uniprot_id_list:
#     download_alphafold_pdb(uniprot_id, save_directory)


Old mess around code to get the mutation counts from an already made csv file, but can be deleted but want to leave it in for inspiration

In [32]:
from Bio.PDB import PDBParser, NeighborSearch
import os

# Specify your list of UniProt IDs and the directory to save the files
mutations_all_proteins_df = pd.read_csv('/Users/davidthomas/Desktop/Research_internship/Determining_interesting_mutations/merged_df_non_sensitive_mutationcounts_all_proteins.csv')
uniprot_id_list = mutations_all_proteins_df['uniprot_id'].unique().tolist()
save_directory = "/Users/davidthomas/Desktop/Research_internship/Determining_interesting_mutations/AF2_structures"  # Update with your desired directory

# Determine the maximum position for each uniprot_id as a proxy for sequence length
max_position_by_uniprot_id = mutations_all_proteins_df.groupby('uniprot_id')['Position'].max()

# Filter the DataFrame for rows where above_threshold == 1
filtered_df = mutations_all_proteins_df[mutations_all_proteins_df['above_threshold'] == 1]

# Group by 'uniprot_id' and collect the 'Position' values into a list for each group
positions_by_uniprot_id = filtered_df.groupby('uniprot_id')['Position'].apply(list)

# Get unique uniprot_id list from the original DataFrame
uniprot_id_list = mutations_all_proteins_df['uniprot_id'].unique().tolist()

# Initialize an empty dictionary to store modified lists
modified_positions = {}

# Iterate over the uniprot_id list and modify positions
for uniprot_id in uniprot_id_list:
    if uniprot_id in positions_by_uniprot_id:  # Check if the uniprot_id exists in positions_by_uniprot_id
        my_list = positions_by_uniprot_id[uniprot_id]
        for i in range(len(my_list)):
            my_list[i] = "A" + str(my_list[i])
        # Store the modified list in the dictionary
        modified_positions[uniprot_id] = my_list

def find_neighbors(pdb_path, target_residues, distance=4.0):
    """
    Find neighboring residues for specified residues in a PDB structure.

    Parameters:
    - pdb_path: Path to the PDB file.
    - target_residues: A list of target residues specified by chain and residue number (e.g., ['A12', 'B45']).
    - distance: Distance threshold to consider an atom as a neighbor (in Ångströms).

    Returns:
    - A dictionary where keys are the specified residues and values are lists of neighboring residues.
    """
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path)

    # Create a list of all atoms in the structure
    atoms = [atom for atom in structure.get_atoms()]

    # Initialize NeighborSearch with all atoms
    neighbor_search = NeighborSearch(atoms)

    neighbors_dict = {}
    for model in structure:
        for chain in model:
            for residue in chain:
                residue_id = f"{chain.id}{residue.get_id()[1]}"
                if residue_id in target_residues:
                    target_atom = residue['CA']  # Use alpha carbon for search
                    neighbors = neighbor_search.search(target_atom.coord, distance, 'R')
                    neighbors_residues = [f"{n.get_parent().id}{n.id[1]}" for n in neighbors if n != residue]
                    neighbors_dict[residue_id] = neighbors_residues
    return neighbors_dict

# Example usage
structure_interface_dict = dict()
folder_path = 'AF2_structures'
for filename in os.listdir(folder_path):
    unique_residues = set()
    if filename.endswith(".pdb"):
        uniprot_id_stripped = filename.strip('.pdb')
        if uniprot_id_stripped == 'P9WI73' or uniprot_id_stripped == 'P9WGW9' or uniprot_id_stripped == 'L7N665':
            continue
        pdb_path = os.path.join(folder_path, filename)
        neighbors = find_neighbors(pdb_path, modified_positions[uniprot_id_stripped])
        # print(f"File: {uniprot_id_stripped}, Neighbours: {neighbors}")
        for x,y in neighbors.items():
            x = x.replace('A','')
            unique_residues.add(x)
            for neighbouring_residue in y:
                neighbouring_residue = int(neighbouring_residue.replace('A', ''))
                unique_residues.add(neighbouring_residue)

        structure_interface_dict[uniprot_id_stripped] = unique_residues
        print('number of interface residues: ' + str(len(modified_positions[uniprot_id_stripped])) + ' number of residues interface + neighbours: ' + str(len(unique_residues)))

#################################################################################
###### OLD BLOCK OF CODE SEPERATED FROM THE NEW BLOCK OF CODE TO AVOID CONFUSION
#################################################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

result_df = pd.read_csv('/Users/davidthomas/Desktop/Research_internship/Determining_interesting_mutations/merged_df_non_sensitive_mutationcounts_all_proteins.csv')

# First, determine the maximum position for each uniprot_id as a proxy for sequence length
max_position_by_uniprot_id = result_df.groupby('uniprot_id')['Position'].max()

# Filter the DataFrame for rows where above_threshold == 1
filtered_df = result_df[result_df['above_threshold'] == 1]

# Group by 'uniprot_id' and collect the 'Position' values into a list for each group
positions_by_uniprot_id = filtered_df.groupby('uniprot_id')['Position'].apply(list)

result_df['above_threshold_structure_neighbours'] = 0

# Then, iterate through the dictionary to update the DataFrame
for uniprot_id, indices in structure_interface_dict.items():
    # Use .loc to update the DataFrame where conditions match
    result_df.loc[(result_df['uniprot_id'] == uniprot_id) & (result_df['Position'].isin(indices)), 'above_threshold_structure_neighbours'] = 1

# First, determine the maximum position for each uniprot_id as a proxy for sequence length
max_position_by_uniprot_id = result_df.groupby('uniprot_id')['Position'].max()

# Filter the DataFrame for rows where above_threshold == 1
filtered_df = result_df[result_df['above_threshold'] == 1]

# Group by 'uniprot_id' and collect the 'Position' values into a list for each group
positions_by_uniprot_id = filtered_df.groupby('uniprot_id')['Position'].apply(list)

positions_and_both_neighbours_by_uniprot_id = dict()

for uniprot_id, positions in positions_by_uniprot_id.items():
    indices_set = set()
    # Retrieve the maximum position (sequence length) for the current uniprot_id
    max_position = max_position_by_uniprot_id[uniprot_id]
    for position in positions:
        # Check and add the current position
        if position > 0:  # Ensuring position is within sequence bounds
            indices_set.add(position)
        # Check and add the previous position if it's greater than 1
        if position > 1:
            indices_set.add(position - 1)
        # Check and add the next position if it does not exceed the sequence length
        if position < max_position:
            indices_set.add(position + 1)
    # Store in the dictionary with the uniprot_id as the key
    positions_and_both_neighbours_by_uniprot_id[uniprot_id] = sorted(list(indices_set))

result_df['above_threshold_neighbours'] = 0

# Then, iterate through the dictionary to update the DataFrame
for uniprot_id, indices in positions_and_both_neighbours_by_uniprot_id.items():
    # Use .loc to update the DataFrame where conditions match
    result_df.loc[(result_df['uniprot_id'] == uniprot_id) & (result_df['Position'].isin(indices)), 'above_threshold_neighbours'] = 1

print(result_df['above_threshold'].value_counts())
print(result_df['above_threshold_neighbours'].value_counts())
print(result_df['above_threshold_structure_neighbours'].value_counts())

result_df


number of interface residues: 42 number of residues interface + neighbours: 134
number of interface residues: 108 number of residues interface + neighbours: 262
number of interface residues: 36 number of residues interface + neighbours: 112
number of interface residues: 68 number of residues interface + neighbours: 186
number of interface residues: 31 number of residues interface + neighbours: 101
number of interface residues: 7 number of residues interface + neighbours: 26
number of interface residues: 13 number of residues interface + neighbours: 39
number of interface residues: 21 number of residues interface + neighbours: 71
number of interface residues: 4 number of residues interface + neighbours: 22
number of interface residues: 52 number of residues interface + neighbours: 144
number of interface residues: 15 number of residues interface + neighbours: 49
number of interface residues: 9 number of residues interface + neighbours: 28
number of interface residues: 30 number of resid

,PIPENN seq,PIPENN predictions,Position,y_preds,above_threshold,uniprot_id,Pipenn_identifiers,A,C,D,...,T,V,W,Y,Total_count,Occurance of mutation for unique samples (%),unique_sample_count,Conservation predictions,above_threshold_structure_neighbours,above_threshold_neighbours
0,M,0.014914,1,0.014914,0,P9WGY9,M_P9WGY9_1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,26380,0.6049,0,0
1,L,0.013961,2,0.013961,0,P9WGY9,L_P9WGY9_2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,26380,0.3791,0,0
2,E,0.013579,3,0.013579,0,P9WGY9,E_P9WGY9_3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,26380,0.4255,0,0
3,G,0.012580,4,0.012580,0,P9WGY9,G_P9WGY9_4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,26380,0.3780,0,0
4,C,0.012439,5,0.012439,0,P9WGY9,C_P9WGY9_5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,26380,0.4699,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25744,K,0.127885,88,0.127885,0,Q6MX01,K_Q6MX01_88,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,9.0,0.282220,3189,0.4019,0,0
25745,D,0.085394,89,0.085394,0,Q6MX01,D_Q6MX01_89,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,3189,0.2612,0,0
25746,A,0.056823,90,0.056823,0,Q6MX01,A_Q6MX01_90,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.094073,3189,0.2224,0,0
25747,V,0.048729,91,0.048729,0,Q6MX01,V_Q6MX01_91,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,3189,0.3104,0,0


Code that can make the mutation count_df for all uniprot_ids and has the ppi residues annotation in the df for PIPENN predicted PPI residues and the sequence and structural neighbours

In [34]:
from Bio import pairwise2
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd

gwas_df = pd.read_csv("/Users/davidthomas/Desktop/Research_internship/Go_nogo/First_pipeline/variants_50k_with_genes.csv")
gwas_df = gwas_df[gwas_df['type'] == 'missense_variant']
# gwas_df = gwas_df[gwas_df['drtype'] != 'Sensitive']
    
def extract_sequences(pipenn_file_path, uniprot_id):    
    # Extract sequence and 'y_preds' from PIPENN file
    pipenn_output_df = pd.read_csv(pipenn_file_path, sep=",")
    current_uniprot_id = pipenn_output_df[pipenn_output_df['prot_id'] == uniprot_id]
    pipenn_output_seq = current_uniprot_id['prot_seq'].iloc[0]
    pipenn_output_seq = pipenn_output_seq.replace(',', '')
    pipenn_output_seq = list(pipenn_output_seq)

    current_uniprot_id['y_preds'] = current_uniprot_id['y_preds'].str.split(',')
    pipenn_output_preds = current_uniprot_id['y_preds'].iloc[0]

    return pipenn_output_seq, pipenn_output_preds

# Function to create an aligned Pandas DataFrame
def create_aligned_dataframe(pipenn_output_seq, pipenn_output_preds, uniprot_id):
    df = pd.DataFrame()
    df['PIPENN seq'] = pipenn_output_seq
    df['PIPENN predictions'] = pipenn_output_preds
    # Reset the index
    df.reset_index(drop=True, inplace=True)
    df['Position'] = df.index + 1

    # Calculate the threshold and add 'above_threshold' column
    df['y_preds'] = pd.to_numeric(df['PIPENN predictions'])
    threshold = 0.345
    df['above_threshold'] = df['y_preds'].apply(lambda x: 1 if x > threshold else 0)
    df['uniprot_id'] = uniprot_id
    # Correctly format 'Pipenn_identifiers' for each row
    df['Pipenn_identifiers'] = df.apply(lambda row: f"{row['PIPENN seq']}_{uniprot_id}_{row['Position']}", axis=1)
    return df


pipenn_file_path = "/Users/davidthomas/Desktop/Research_internship/Go_nogo/First_pipeline/PIPENN_output/preds-UserDS_P.csv"
uniprot_id_df = pd.read_csv('/Users/davidthomas/Desktop/Research_internship/Go_nogo/First_pipeline/locus_uniprot_gene.csv', delimiter=';')
uniprot_id_list = uniprot_id_df['UniProt_ID'].tolist()
all_uniprot_df = pd.DataFrame()
for uniprot_id in uniprot_id_list:
# Extract sequences
    pipenn_output_seq, pipenn_output_preds = extract_sequences(pipenn_file_path, uniprot_id)

    # Create aligned Pandas DataFrame
    aligned_df = create_aligned_dataframe(pipenn_output_seq, pipenn_output_preds, uniprot_id)
    all_uniprot_df = pd.concat([all_uniprot_df, aligned_df])

    # Print the DataFrame

# all_uniprot_df[all_uniprot_df['uniprot_id'] == 'Q6MX01']['y_preds']
all_uniprot_df[all_uniprot_df['above_threshold'] == 1]['uniprot_id'].value_counts()
def three_to_one_letter(codes):
    AA_DICT = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G',
        'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
        'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V',
        'TRP': 'W', 'TYR': 'Y'
    }
    return AA_DICT.get(codes, 'X')

# Assuming you have these DataFrames defined:

def make_mutation_df(uniprot_id, gwas_df, gene):


    filtered_on_gene_df = gwas_df[gwas_df['gene'] == gene]
    unique_sample_count = filtered_on_gene_df['sample_id'].nunique()
    id_df_with_genes = pd.DataFrame()

    id_df_with_genes['original_aa'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.([A-Za-z]+)')[0].str.upper()
    id_df_with_genes['position'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.[A-Za-z]+(\d+)')
    # id_df_with_genes['changed_into_aa'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.[A-Za-z]+(\d+)([A-Za-z]{3})')[0].str.upper()
    id_df_with_genes['changed_into_aa'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.[A-Za-z]+\d+([A-Za-z]{3})')[0].str.upper()
    id_df_with_genes['single_letter_original_aa'] = id_df_with_genes['original_aa'].apply(three_to_one_letter)
    id_df_with_genes['single_letter_changed_aa'] = id_df_with_genes['changed_into_aa'].apply(three_to_one_letter)
    id_df_with_genes['Pipenn_identifiers'] = id_df_with_genes['single_letter_original_aa'] + f"_{uniprot_id}_" + id_df_with_genes['position']

    # Group by 'Pipenn_identifiers' and 'single_letter_changed_aa', count the occurrences, unstack the result
    count_df = id_df_with_genes.groupby(['Pipenn_identifiers', 'single_letter_changed_aa']).size().unstack(fill_value=0)

    # Add columns for all possible single-letter amino acids if you want every possible mutation to be represented even if it doesn't occur
    possible_aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    for aa in possible_aa:
        if aa not in count_df.columns:
            count_df[aa] = 0

    # Sort the columns if you want them in a specific order
    count_df = count_df[possible_aa]
    # Reset index to make 'Pipenn_identifiers' a column again if needed
    count_df.reset_index(inplace=True)
    count_df['Total_count'] = count_df[possible_aa].sum(axis=1)
    count_df['Occurance of mutation for unique samples (%)'] = (count_df['Total_count'] / unique_sample_count) * 100

    return count_df, unique_sample_count

if __name__ == '__main__':
    locus_uniprot_gene_df = pd.read_csv('/Users/davidthomas/Desktop/Research_internship/Go_nogo/First_pipeline/locus_uniprot_gene.csv', delimiter=';')
    uniprot_id_list = locus_uniprot_gene_df['UniProt_ID'].tolist()
    Gene_name_list = locus_uniprot_gene_df['Gene_name'].tolist()
    Full_mutation_count_df = pd.DataFrame()
    unique_sample_count_dict = {}
    for uniprot_id, gene_name in zip(uniprot_id_list, Gene_name_list):
        mutation_count_df, unique_sample_count = make_mutation_df(uniprot_id, gwas_df, gene_name)
        unique_sample_count_dict[uniprot_id] = unique_sample_count
        Full_mutation_count_df = pd.concat([Full_mutation_count_df, mutation_count_df])

    pipenn_and_mutations = pd.merge(all_uniprot_df, Full_mutation_count_df, on='Pipenn_identifiers', how='left')
    pipenn_and_mutations.fillna(0, inplace=True)
    pipenn_and_mutations['unique_sample_count'] = pipenn_and_mutations['uniprot_id'].map(unique_sample_count_dict)

import os
import pandas as pd

def get_uniprot_id_from_file_name(file_name):
    """
    Extracts the UniProt ID from a file name.
    Assumes file name format "UniProtID.extension"
    """
    uniprot_id = file_name.split('.')[0]
    return uniprot_id

def read_csv_files_in_folders(root_folder):
    # Initialize DataFrame with correct column names
    conservation_data = pd.DataFrame(columns=['seq', 'conservation_score', 'uniprot_id', 'Pipenn_identifiers'])
    
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".csv"):
                uniprot_id = get_uniprot_id_from_file_name(file)
                file_path = os.path.join(subdir, file)
                df = pd.read_csv(file_path, header=None)
                df.columns = ['seq', 'conservation_score']  # Assuming the CSV has two columns
                df['uniprot_id'] = uniprot_id
                
                # Assuming 'seq' is in the first column (now named 'seq'), and you want to add a unique identifier
                df['Pipenn_identifiers'] = df.index.to_series().apply(lambda x: f"{df.at[x, 'seq']}_{uniprot_id}_{x + 1}")
                
                conservation_data = pd.concat([conservation_data, df], ignore_index=True)
    
    return conservation_data

# Adjust the root_folder path as necessary for your environment
root_folder = '/Users/davidthomas/Desktop/Research_internship/Embedding_conservation/kibby/outpu'
conservation_data = read_csv_files_in_folders(root_folder)

fasta_based_conservation = pd.read_csv('/Users/davidthomas/Downloads/output-7')

unique_uniprot_id_list = fasta_based_conservation['header'].unique()

fasta_conservation_df = pd.DataFrame(columns = ['Pipenn_identifiers', 'Conservation predictions'])

for uniprot_id in unique_uniprot_id_list:
    filtered_df = fasta_based_conservation[fasta_based_conservation['header'] == uniprot_id]
    # Extract sequences and predictions
    uniprot_seq = filtered_df['sequence'].iloc[0]
    # Convert the list of string representations to integers
    conserv_preds_list = filtered_df['conservation'].str.split(' ').iloc[0]
    connserv_preds = [float(pred) for pred in conserv_preds_list]


    # Generate PiPeNN identifiers
    pipenn_identifiers = [f"{seq}_{uniprot_id}_{i+1}" for i, seq in enumerate(uniprot_seq)]

    # Create and return DataFrame
    result_df = pd.DataFrame({
        'Pipenn_identifiers': pipenn_identifiers,
        'Conservation predictions': connserv_preds
    })

    fasta_conservation_df = pd.concat([fasta_conservation_df, result_df])

result_df = pd.merge(pipenn_and_mutations, fasta_conservation_df, on = 'Pipenn_identifiers', how = 'left')
result_df

from Bio.PDB import PDBParser, NeighborSearch
import os
import numpy as np
import pandas as pd

def modify_positions(filtered_df):
    # Group by 'uniprot_id' and collect the 'Position' values into a list for each group
    positions_by_uniprot_id = filtered_df.groupby('uniprot_id')['Position'].apply(list)

    # Initialize an empty dictionary to store modified lists
    modified_positions = {}

    # Iterate over the uniprot_id list and modify positions
    for uniprot_id in uniprot_id_list:
        if uniprot_id in positions_by_uniprot_id:  
            my_list = positions_by_uniprot_id[uniprot_id]
            for i in range(len(my_list)):
                my_list[i] = "A" + str(my_list[i])
            modified_positions[uniprot_id] = my_list
    
    return modified_positions

def find_neighbors(pdb_path, target_residues, distance=4.0):
    """
    Find neighboring residues for specified residues in a PDB structure.

    Parameters:
    - pdb_path: Path to the PDB file.
    - distance: Distance threshold to consider an atom as a neighbor (in Ångströms).

    Returns:
    - A dictionary where keys are the specified residues and values are lists of neighboring residues.
    """
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path)

    # Create a list of all atoms in the structure
    atoms = [atom for atom in structure.get_atoms()]

    # Initialize NeighborSearch with all atoms
    neighbor_search = NeighborSearch(atoms)

    neighbors_dict = {}
    for model in structure:
        for chain in model:
            for residue in chain:
                residue_id = f"{chain.id}{residue.get_id()[1]}"
                if residue_id in target_residues:
                    target_atom = residue['CA']  # Use alpha carbon for search
                    neighbors = neighbor_search.search(target_atom.coord, distance, 'R')
                    neighbors_residues = [f"{n.get_parent().id}{n.id[1]}" for n in neighbors if n != residue]
                    neighbors_dict[residue_id] = neighbors_residues
    return neighbors_dict

def get_structure_interface_dict(folder_path, modified_positions):
    structure_interface_dict = {}
    for filename in os.listdir(folder_path):
        unique_residues = set()
        if filename.endswith(".pdb"):
            uniprot_id_stripped = filename.strip('.pdb')
            if uniprot_id_stripped in ['P9WI73', 'P9WGW9', 'L7N665']:
                continue
            pdb_path = os.path.join(folder_path, filename)
            neighbors = find_neighbors(pdb_path, modified_positions[uniprot_id_stripped])
            for x,y in neighbors.items():
                x = x.replace('A','')
                unique_residues.add(x)
                for neighbouring_residue in y:
                    neighbouring_residue = int(neighbouring_residue.replace('A', ''))
                    unique_residues.add(neighbouring_residue)

            structure_interface_dict[uniprot_id_stripped] = unique_residues
            print('number of interface residues: ' + str(len(modified_positions[uniprot_id_stripped])) + ' number of residues interface + neighbours: ' + str(len(unique_residues)))
    
    return structure_interface_dict

def update_dataframe(result_df, structure_interface_dict):
    result_df['above_threshold_structure_neighbours'] = 0

    for uniprot_id, indices in structure_interface_dict.items():
        result_df.loc[(result_df['uniprot_id'] == uniprot_id) & (result_df['Position'].isin(indices)), 'above_threshold_structure_neighbours'] = 1

    max_position_by_uniprot_id = result_df.groupby('uniprot_id')['Position'].max()
    filtered_df = result_df[result_df['above_threshold'] == 1]
    positions_by_uniprot_id = filtered_df.groupby('uniprot_id')['Position'].apply(list)
    positions_and_both_neighbours_by_uniprot_id = {}
    for uniprot_id, positions in positions_by_uniprot_id.items():
        indices_set = set()
        max_position = max_position_by_uniprot_id[uniprot_id]
        for position in positions:
            if position > 0:
                indices_set.add(position)
            if position > 1:
                indices_set.add(position - 1)
            if position < max_position:
                indices_set.add(position + 1)
        positions_and_both_neighbours_by_uniprot_id[uniprot_id] = sorted(list(indices_set))

    result_df['above_threshold_neighbours'] = 0

    for uniprot_id, indices in positions_and_both_neighbours_by_uniprot_id.items():
        result_df.loc[(result_df['uniprot_id'] == uniprot_id) & (result_df['Position'].isin(indices)), 'above_threshold_neighbours'] = 1

    return result_df


uniprot_id_list = result_df['uniprot_id'].unique().tolist()  # Update with your desired directory

filtered_df = result_df[result_df['above_threshold'] == 1]
modified_positions = modify_positions(filtered_df)
folder_path = 'AF2_structures'
structure_interface_dict = get_structure_interface_dict(folder_path, modified_positions)
result_df = update_dataframe(result_df, structure_interface_dict)

print(result_df['above_threshold'].value_counts())
print(result_df['above_threshold_neighbours'].value_counts())
print(result_df['above_threshold_structure_neighbours'].value_counts())

result_df



/var/folders/m8/vf66sd1s2y54tllwvc5f3ksw0000gn/T/ipykernel_29734/1249922840.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_uniprot_id['y_preds'] = current_uniprot_id['y_preds'].str.split(',')
/var/folders/m8/vf66sd1s2y54tllwvc5f3ksw0000gn/T/ipykernel_29734/1249922840.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_uniprot_id['y_preds'] = current_uniprot_id['y_preds'].str.split(',')
/var/folders/m8/vf66sd1s2y54tllwvc5f3ksw0000gn/T/ipykernel_29734/1249922840.py:18: SettingWithCopyWa

number of interface residues: 42 number of residues interface + neighbours: 134
number of interface residues: 108 number of residues interface + neighbours: 262
number of interface residues: 36 number of residues interface + neighbours: 112
number of interface residues: 68 number of residues interface + neighbours: 186
number of interface residues: 31 number of residues interface + neighbours: 101
number of interface residues: 7 number of residues interface + neighbours: 26
number of interface residues: 13 number of residues interface + neighbours: 39
number of interface residues: 21 number of residues interface + neighbours: 71
number of interface residues: 4 number of residues interface + neighbours: 22
number of interface residues: 52 number of residues interface + neighbours: 144
number of interface residues: 15 number of residues interface + neighbours: 49
number of interface residues: 9 number of residues interface + neighbours: 28
number of interface residues: 30 number of resid

,PIPENN seq,PIPENN predictions,Position,y_preds,above_threshold,uniprot_id,Pipenn_identifiers,A,C,D,...,T,V,W,Y,Total_count,Occurance of mutation for unique samples (%),unique_sample_count,Conservation predictions,above_threshold_structure_neighbours,above_threshold_neighbours
0,M,0.014913763125093482,1,0.014914,0,P9WGY9,M_P9WGY9_1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,16523,0.6049,0,0
1,L,0.01396083755180989,2,0.013961,0,P9WGY9,L_P9WGY9_2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,16523,0.3791,0,0
2,E,0.01357896622067659,3,0.013579,0,P9WGY9,E_P9WGY9_3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,16523,0.4255,0,0
3,G,0.0125803182434193,4,0.012580,0,P9WGY9,G_P9WGY9_4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,16523,0.3780,0,0
4,C,0.012439226726744803,5,0.012439,0,P9WGY9,C_P9WGY9_5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,16523,0.4699,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25744,K,0.12788485442403058,88,0.127885,0,Q6MX01,K_Q6MX01_88,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,9.0,2.616279,344,0.4019,0,0
25745,D,0.08539391011069175,89,0.085394,0,Q6MX01,D_Q6MX01_89,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,344,0.2612,0,0
25746,A,0.056822860473660565,90,0.056823,0,Q6MX01,A_Q6MX01_90,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.290698,344,0.2224,0,0
25747,V,0.048728549027039846,91,0.048729,0,Q6MX01,V_Q6MX01_91,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,344,0.3104,0,0


This code extracts unique single, double, and multiple mutations from the gwas_df per protine for each sample that contains this mutation. It then generates a unique mutation profile for each sample, arranging them based on their occurrence frequency. These profiles can be utilised with ddMut for stability prediction on the structure.

In [1]:
from itertools import combinations
from collections import Counter
import os
import pandas as pd


def three_to_one_letter(codes):
    AA_DICT = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G',
        'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
        'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V',
        'TRP': 'W', 'TYR': 'Y'
    }
    return AA_DICT.get(codes, 'X')

# Assuming you have these DataFrames defined:

def make_mutation_df(uniprot_id, gwas_df):
    filtered_on_gene_df = gwas_df[gwas_df['UniProt_ID'] == uniprot_id]
    sample_id_list = filtered_on_gene_df['sample_id'].unique().tolist()
    id_df_with_genes = pd.DataFrame()

    id_df_with_genes['sample_id'] = filtered_on_gene_df['sample_id']
    id_df_with_genes['original_aa'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.([A-Za-z]+)')[0].str.upper()
    id_df_with_genes['position'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.[A-Za-z]+(\d+)')
    # id_df_with_genes['changed_into_aa'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.[A-Za-z]+(\d+)([A-Za-z]{3})')[0].str.upper()
    id_df_with_genes['changed_into_aa'] = filtered_on_gene_df['protein_change'].str.extract(r'p\.[A-Za-z]+\d+([A-Za-z]{3})')[0].str.upper()
    id_df_with_genes['single_letter_original_aa'] = id_df_with_genes['original_aa'].apply(three_to_one_letter)
    id_df_with_genes['single_letter_changed_aa'] = id_df_with_genes['changed_into_aa'].apply(three_to_one_letter)
    # id_df_with_genes['Pipenn_identifiers'] = id_df_with_genes['single_letter_original_aa'] + f"_{uniprot_id}_" + id_df_with_genes['position']
    id_df_with_genes['Pipenn_identifiers_change'] = id_df_with_genes['single_letter_original_aa'] + f"_{uniprot_id}_" + id_df_with_genes['position'] + id_df_with_genes['single_letter_changed_aa']

    return id_df_with_genes

def get_combinations(df, number_of_combinations):
    ''' For each sample, get all possible combinations of mutations '''

    combinations_list = []
    for sample_id, group in df.groupby('sample_id'):
        mutations = group['Pipenn_identifiers_change'].tolist()
        if len(mutations) >= number_of_combinations:
            combinations_list.extend(combinations(mutations, number_of_combinations))
    return combinations_list

def get_top_combinations(combinations_list, top_n=20):
    ''' Get the top n most common combinations of mutations and return them as a list of tuples'''
    # Count the occurrences of each combination using Counter
    combination_counts = Counter(tuple(comb) for comb in combinations_list)
    # Sort the combinations by their occurrence count in descending order
    sorted_combinations = sorted(combination_counts.items(), key=lambda x: x[1], reverse=True)
    # Select the top n combinations
    top_combinations = sorted_combinations[:top_n]

    return top_combinations

def get_mutations_from_combination(combination):
    ''' Get the mutations from the list of combinations that occur the most'''

    mutation_list = []
    for x in range(len(combination)):
        mutation_line = ''
        for mutation in combination[x][0]:
            a = mutation.split('_')[0] + mutation.split('_')[2]
            mutation_line += 'A ' + a + '; '
        mutation_list.append(mutation_line[:-2])  # Remove the extra space and semicolon at the end 
    return mutation_list

def make_mutation_txt_file(bi_count_mutation_list, tri_count_mutation_list, uniprot_id):
    ''' Make a txt file with the mutations that occur the most'''
    file_path = 'full_list_multiple_mutations'

    os.makedirs(file_path, exist_ok=True)
    file_path = os.path.join(file_path, f'{uniprot_id}_output.txt')
    with open(file_path, 'w') as f:
        for mutation in tri_count_mutation_list:
            f.write(mutation + '\n')
        for mutation in bi_count_mutation_list:
            f.write(mutation + '\n')


def main(uniprot_id, gwas_df):
    count_df = make_mutation_df(uniprot_id, gwas_df)
    bi_combo_count_df = get_combinations(count_df, 2)
    tri_combo_count_df = get_combinations(count_df, 3)
    bi_count = get_top_combinations(bi_combo_count_df, 10000)
    tri_count = get_top_combinations(tri_combo_count_df, 10000)
    print(f"Top 10 combinations for {uniprot_id}")
    print("With 2 mutations:")
    print(get_top_combinations(bi_combo_count_df, 10))
    print(f"With 3 mutations:")
    print(get_top_combinations(tri_combo_count_df, 10))
    bi_count_mutation_list = get_mutations_from_combination(bi_count)
    tri_count_mutation_list = get_mutations_from_combination(tri_count)

    return bi_count_mutation_list, tri_count_mutation_list


gwas_df = gwas_df = pd.read_csv('/Users/davidthomas/Desktop/Research_internship/Go_nogo/First_pipeline/variants_50k_with_genes.csv')
gwas_df = gwas_df[gwas_df['type'] == 'missense_variant']
# gwas_df = gwas_df[gwas_df['drtype'] != 'Sensitive']

unique_uniprot_ids = gwas_df['UniProt_ID'].unique()
uniprot_id_list = unique_uniprot_ids.tolist()

# I want to calculate the number of samples that have a certain combination of different mutations within a sample. 
# for uniprot_id in uniprot_id_list:
for uniprot_id in uniprot_id_list:
    x, y = main(uniprot_id, gwas_df)
    make_mutation_txt_file(x, y, uniprot_id)

import pandas as pd
import os
from itertools import combinations
from collections import Counter

def main(uniprot_id, gwas_df):
    count_df = make_mutation_df(uniprot_id, gwas_df)

    # Get top single mutations
    single_mutations = count_df['Pipenn_identifiers_change'].tolist()
    single_count = Counter(single_mutations).most_common(10000)
    single_count_mutation_list = [mutation[0] for mutation in single_count]
    single_count_mutation_list = ['A ' + x.split('_')[0] + x.split('_')[2] for x in single_count_mutation_list]


    # Combine single, double, and triple mutations
    all_mutations = single_count_mutation_list

    # Create a new folder to store output files
    folder_path = 'full_list_single_mutation'
    os.makedirs(folder_path, exist_ok=True)

    # Write mutations to a text file
    file_path = os.path.join(folder_path, f'{uniprot_id}_output.txt')
    with open(file_path, 'w') as f:
        for mutation in all_mutations:
            f.write(mutation + '\n')

    return all_mutations

# Assuming gwas_df is defined
unique_uniprot_ids = gwas_df['UniProt_ID'].unique().tolist()

for uniprot_id in unique_uniprot_ids:
    mutations = main(uniprot_id, gwas_df)
    print(f"Top mutations for {uniprot_id}:")
    print(mutations)

#################################################################################
###### This is where the single mutation list txt file is created
#################################################################################

import pandas as pd
import os
from itertools import combinations
from collections import Counter

def main(uniprot_id, gwas_df):
    count_df = make_mutation_df(uniprot_id, gwas_df)

    # Get top single mutations
    single_mutations = count_df['Pipenn_identifiers_change'].tolist()
    single_count = Counter(single_mutations).most_common(10000)
    single_count_mutation_list = [mutation[0] for mutation in single_count]
    single_count_mutation_list = ['A ' + x.split('_')[0] + x.split('_')[2] for x in single_count_mutation_list]


    # Combine single, double, and triple mutations
    all_mutations = single_count_mutation_list

    # Create a new folder to store output files
    folder_path = 'full_list_single_mutation'
    os.makedirs(folder_path, exist_ok=True)

    # Write mutations to a text file
    file_path = os.path.join(folder_path, f'{uniprot_id}_output.txt')
    with open(file_path, 'w') as f:
        for mutation in all_mutations:
            f.write(mutation + '\n')

    return all_mutations

# Assuming gwas_df is defined
unique_uniprot_ids = gwas_df['UniProt_ID'].unique().tolist()

for uniprot_id in unique_uniprot_ids:
    mutations = main(uniprot_id, gwas_df)
    print(f"Top mutations for {uniprot_id}:")
    print(mutations)

Top 10 combinations for P9WGY9
With 2 mutations:
[(('S_P9WGY9_450L', 'E_P9WGY9_761D'), 249), (('S_P9WGY9_388L', 'E_P9WGY9_639D'), 170), (('S_P9WGY9_450L', 'L_P9WGY9_731P'), 143), (('D_P9WGY9_435G', 'L_P9WGY9_452P'), 106), (('S_P9WGY9_450L', 'R_P9WGY9_827C'), 96), (('D_P9WGY9_435G', 'I_P9WGY9_1106T'), 95), (('L_P9WGY9_452P', 'I_P9WGY9_1106T'), 95), (('S_P9WGY9_450L', 'V_P9WGY9_695L'), 73), (('S_P9WGY9_450L', 'A_P9WGY9_692T'), 69), (('S_P9WGY9_450L', 'Q_P9WGY9_409R'), 59)]
With 3 mutations:
[(('D_P9WGY9_435G', 'L_P9WGY9_452P', 'I_P9WGY9_1106T'), 95), (('T_P9WGY9_350I', 'S_P9WGY9_388L', 'E_P9WGY9_639D'), 35), (('Q_P9WGY9_432P', 'K_P9WGY9_799Q', 'R_P9WGY9_827H'), 9), (('S_P9WGY9_239N', 'T_P9WGY9_347P', 'V_P9WGY9_355I'), 5), (('S_P9WGY9_239N', 'V_P9WGY9_243T', 'R_P9WGY9_253M'), 5), (('S_P9WGY9_239N', 'V_P9WGY9_243T', 'V_P9WGY9_262A'), 5), (('S_P9WGY9_239N', 'R_P9WGY9_253M', 'V_P9WGY9_262A'), 5), (('V_P9WGY9_243T', 'R_P9WGY9_253M', 'V_P9WGY9_262A'), 5), (('D_P9WGY9_190E', 'S_P9WGY9_239N', 'V

This segment of the code is designed to fetch the unique mutations from the ddMut predictions. It leverages the job_id list obtained for each mutation impact prediction, derived from the AF2 structures. This job_id list is in a csv file that has been made manually bu using the ddMut server on: https://biosig.lab.uq.edu.au/ddmut/ and the mutations txt files used to predict the structural change on are made in the code block above.

In [ ]:
# make a quick change for commit

Here we make the counts of the unique mutation profiles, so we can see which mutations are important, look into whether you want to select only mutations that have a resistant or sensitive phenotype or just mutations in general. 